## Load Packages

In [150]:
suppressWarnings({
  # Code block with multiple lines
  library(DT)
  library(nlme)
  library(gridExtra)
  library(data.table)
  library(plyr)
  library(dplyr)
  library(formattable)
  library(tidyr)
  library(MASS) 
  library(shinyjs)
  library(compiler)
  library(future.apply)
  library(PowerAnalysisIL)
  library(tictoc)
  library(viridis)
})



## Power Estimation

$$\pi = P(Rejecting H_0|H_0 is false)$$

Procedure:
1. Sepecify the Model
2. Define Parameters
3. Simulate Data
4. Test Hypothesis
5. Repeat Step2 and 3 Many Times
6. Calculate the percentage of which the null hypothesis being rejected 

## The Model

**Model 3:**
$$y_{ij}=b_{0i} +b_{1i}*|O_{ij}|+ b_{2i}*PE_{ij}+ b_{3i}*PE_{ij}*|O_{ij}| + b_{i}*y_{i(j-1)}+\epsilon_{ij}, \epsilon_{ij} \sim N(0,\sigma_{i}^2)$$

$$b_{0i} \sim N(B_0, \sigma_0^2)$$
$$b_{1i} \sim N(B_1, \sigma_1^2)$$
$$b_{2i} \sim N(B_2, \sigma_2^2)$$
$$b_{3i} \sim N(B_3, \sigma_3^2)$$
$$b_{i} \sim N(B, \sigma^2)$$

## Set the Number of Participants and Observations

In [151]:
N = 10 # Vector with the number of participants for which compute power
T.obs = 192# Number of trials for each individual
alpha = 0.05 # Significant level
side.test = 1
Opt.Method = 'REML' # Set the optimization method for lme 
R = 40 # Number of Monte Carlo replicates

## Set Predictors

In [152]:
#set predictors
outcome.true = c(rep(-2,2*24), rep(-1,2*24), rep(1,2*24),rep(2,2*24))
outcome.win = c(rep(c(rep(1,12),rep(2,12)),4),rep(1,2*24),rep(2,2*24))
outcome.loss = c(rep(-2,2*24), rep(-1,2*24),rep(c(rep(-1,12),rep(-2,12)),4))
prob.win = rep(c(rep(0,24),rep(1,24)),4)
prob.loss = rep(c(rep(1,24),rep(0,24)),4)

expectedness = c()
outcome.dummy = c()
outcome.absolute = abs(outcome.true)

for (i in 1:length(outcome.true)){
  if (outcome.true[i]<0){
    expectedness[i] = prob.win[i]
    outcome.dummy[i] = -1
  }else{
    expectedness[i] = prob.loss[i]
    outcome.dummy[i] = 1
  }
}

expectedness = rep(expectedness, times = N)
outcome.dummy = rep(outcome.dummy, times = N)
outcome.absolute = rep(outcome.absolute, times = N)
outcome.true = rep(outcome.true, times = N)

condition = cbind(outcome.dummy, outcome.absolute, outcome.true, outcome.win, outcome.loss, prob.win, prob.loss, expectedness)
random_indices = sample(nrow(condition))
condition = data.frame(condition[random_indices, ])


## Input Information from the Pilot Study

In [153]:
# Fixed effects
b00 = 50 # Fixed intercept
b01 = 1.1 # Effect of the Level 1 GR
b10 = 0.16 # Fixed slope Level 1 PE
b20 = 0.16 # Fixed slope Level 1 GR*PE
b11 = 0.05 # Fixed slop Level 1 AR

# Distribution Level 1 errors
sigma = 4 # Std. deviation of the Level 1 error
rho = 0 # Autocorrelation of the Level 1 error

# Distribution random effects
sigma.v0 = 5.45 # Std. deviation of the random intercept
sigma.v1 = 0.5 # Std. deviation of the random slope GR
sigma.v2 = 0.5 # Std. deviation of the random slope PE
sigma.v3 = 0.5 # Std. deviation of the random slope GR*PE
sigma.v4 = 0.5 # Std. deviation of the random slope AR

rho.v01 = 0.156 # correlation between the random intercept and the random slope (GR)
rho.v02 = 0.156 # correlation between the random intercept and the random slope (PE)
rho.v03 = 0.156 # correlation between the random intercept and the random slope (GR*PE)
rho.v04 = 0.156 # correlation between the random intercept and the random slope (AR)

rho.v12 = 0.156 # correlation between the random slope (GR) and the random slope (PE)
rho.v13 = 0.156 # correlation between the random slope (GR) and the random slope (GR*PE)
rho.v14 = 0.156 # correlation between the random slope (GR) and the random slope (AR)

rho.v23 = 0.156 # correlation between the random slope (PE) and the random slope (GR*PE)
rho.v24 = 0.156 # correlation between the random slope (PE) and the random slope (AR)

rho.v34 = 0.156 # correlation between the random slope (GR*PE) and the random slope (AR)



## Data Simulation

In [158]:
Sim.Data.ML.5.VAR = function(N,T.obs,
                             b00,b01,b10,b11,b20,
                             sigma,
                             sigma.v0,sigma.v1,sigma.v2,sigma.v3,sigma.v4,
                             rho.v01,rho.v02,rho.v03,rho.v04,
                             rho.v12,rho.v13,rho.v14,
                             rho.v23,rho.v24,
                             rho.v34){
# Total number of subjects
N = N

# Create variables days, beeps per day and Z
data.IL = expand.grid(Time=1:T.obs)

# Create variable subjno
subjno = expand.grid(1:T.obs,1:N)[,2]
data.IL = cbind(subjno,data.IL)

# Simulate error level-1 (population-level error)
# this is the line that's being used
E = rnorm(T.obs*N,0,sigma)

# Simulate error level-2
# Simulate between-subject random effect
# build the variance-covariance matrix
Sigma.v = diag(c(sigma.v0^2,sigma.v1^2,sigma.v2^2,sigma.v3^2,sigma.v4^2))
Sigma.v[lower.tri(Sigma.v)] = c(rho.v01*sigma.v0*sigma.v1,
                                rho.v02*sigma.v0*sigma.v2,
                                rho.v03*sigma.v0*sigma.v3,
                                rho.v04*sigma.v0*sigma.v4,
                                rho.v12*sigma.v1*sigma.v2,
                                rho.v13*sigma.v1*sigma.v3,
                                rho.v14*sigma.v1*sigma.v4,
                                rho.v23*sigma.v2*sigma.v3,
                                rho.v24*sigma.v2*sigma.v4,
                                rho.v34*sigma.v3*sigma.v4)
Sigma.v[upper.tri(Sigma.v)] = c(rho.v01*sigma.v0*sigma.v1,
                                rho.v02*sigma.v0*sigma.v2,
                                rho.v03*sigma.v0*sigma.v3,
                                rho.v04*sigma.v0*sigma.v4,
                                rho.v12*sigma.v1*sigma.v2,
                                rho.v13*sigma.v1*sigma.v3,
                                rho.v14*sigma.v1*sigma.v4,
                                rho.v23*sigma.v2*sigma.v3,
                                rho.v24*sigma.v2*sigma.v4,
                                rho.v34*sigma.v3*sigma.v4)

if (eigen(Sigma.v)$values[2] <= 0) {stop('The covariance matrix of the level-2 errors must be positive definite')}

# simulate the participant-wise coefficents
V.i = mvrnorm(N,rep(0,ncol(Sigma.v)),Sigma.v)

V = matrix(0,T.obs*N,ncol(V.i))

#true parameters
for (i in 1:N){
  V[which(data.IL$subjno==i),] = matrix(rep(V.i[i,], each = T.obs), ncol = T.obs)
}

# Set parameters and variables
B00 = rep(b00,nrow(data.IL))
B01 = rep(b01,nrow(data.IL))
B10 = rep(b10,nrow(data.IL))
B11 = rep(b11,nrow(data.IL))
B20 = rep(b20,nrow(data.IL))


Y = rep(NA, length(subjno))
Y_previous = rep(NA, length(subjno))
Y_previous[sapply(1:10, function(x) which(subjno == x)[1])] = 0

for (i in 1:length(subjno)){
  Y[i] = B00[i] + B01[i]*condition$outcome.absolute[i] + B10[i]*condition$expectedness[i] + B11[i]*condition$outcome.absolute[i]*condition$expectedness[i] +
    B20[i]*Y_previous[i] + V[i,1] + V[i,2]*condition$outcome.absolute[i] + V[i,3]*condition$expectedness[i] + V[i,4]*condition$outcome.absolute[i]*condition$expectedness[i]
  + V[i,5]*Y_previous[i] + E[i]
  if (is.na(Y_previous[i+1]) & i<length(subjno)) {
    Y_previous[i+1] = Y[i]
  }else{}
}



                  
data = data.frame(cbind(data.IL,Y,condition$expectedness,condition$outcome.absolute,condition$outcome.dummy,condition$outcome.true,Y_previous))

}  

## Fit the Simulated Data Back

In [159]:
## This function uses Monte Carlo simulations for computing standard errors and statistical power. 

Power.Simulation.ML.5.VAR = function(data,T.obs,
                                     b00,b01,b10,b11,b20,
                                     sigma,
                                     sigma.v0,sigma.v1,sigma.v2,sigma.v3,sigma.v4,
                                     rho.v01,rho.v02,rho.v03,rho.v04,
                                     rho.v12,rho.v13,rho.v14,
                                     rho.v23,rho.v24,
                                     rho.v34,
                                     alpha,
                                     side.test,
                                     Opt.Method){
  # Fit linear mixed-effects models
  
  if (Opt.Method == 'ML'){
    # Maximum Likelihood
    fit.lme = try(lme(Y ~ outcome.absolute + expectedness + outcome.absolute*expectedness + Y_previous, random = ~ 1 + outcome.absolute + expectedness + outcome.absolute*expectedness + Y_previous|subjno,correlation = corAR1(),data=data,na.action=na.omit,method='ML',
                      control=lmeControl(opt='optim')),silent = FALSE)
  }
  
  if (Opt.Method == 'REML'){
    fit.lme = try(lme(Y ~ outcome.absolute + expectedness + outcome.absolute*expectedness + Y_previous, random = ~ 1 + outcome.absolute + expectedness + outcome.absolute*expectedness + Y_previous|subjno,correlation = corAR1(),data=data,na.action=na.omit,method='REML',
                      control=lmeControl(opt='optim')),silent = FALSE)
  }
  
  if (length(fit.lme)>1){
    
    # Obtain the estimated coefficients of the model
    beta.hat.lme = coef(summary(fit.lme))[,'Value']
    
    # Obtain the standard errors
    StdError.beta.lme = coef(summary(fit.lme))[,'Std.Error']
    
    # Compute power and standard error from lme  
    
    if (side.test == 1){ # One-side test: positive
      p.value = pt(coef(summary(fit.lme))[,4], coef(summary(fit.lme))[,3], lower = FALSE)
    }
    
    if (side.test == 2){ # One-side test: negative
      p.value = pt(coef(summary(fit.lme))[,4], coef(summary(fit.lme))[,3], lower = TRUE)
      
    }
    
    if (side.test == 3){ # Two-tailed test
      p.value = 2*pt(-abs(coef(summary(fit.lme))[,4]), coef(summary(fit.lme))[,3])
    }
    
    power.hat.lme = p.value < alpha
    
    return(list(beta.hat.lme=beta.hat.lme,
                power.hat.lme=power.hat.lme,
                StdError.beta.lme=StdError.beta.lme,
                p.value=p.value))}
  
  if (length(fit.lme)==1){
    return(list(fit.lme))
  }}


#############################
######Power Analysis#########
Power.Simulation.Estimates.ML.5.VAR = function(N,T.obs,
                                               b00,b01,b10,b11,b20,
                                               sigma,
                                               sigma.v0,sigma.v1,sigma.v2,sigma.v3,sigma.v4,
                                               rho.v01,rho.v02,rho.v03,rho.v04,
                                               rho.v12,rho.v13,rho.v14,
                                               rho.v23,rho.v24,
                                               rho.v34,
                                               alpha,
                                               side.test,
                                               Opt.Method,R){
  
  tic()
  # Simulate data from the linear mixed-effects model  
  data = lapply(1:R, function(r) 
    Sim.Data.ML.5.VAR(N,T.obs,
                      b00,b01,b10,b11,b20,
                      sigma,
                      sigma.v0,sigma.v1,sigma.v2,sigma.v3,sigma.v4,
                      rho.v01,rho.v02,rho.v03,rho.v04,
                      rho.v12,rho.v13,rho.v14,
                      rho.v23,rho.v24,
                      rho.v34)) 
  
  # Simulation-based power analysis using Monte Carlo simulation
  fit.list.sim = lapply(1:R, function(r) Power.Simulation.ML.5.VAR(data[[r]],T.obs,
                                                                   b00,b01,b10,b11,b20,
                                                                   sigma,
                                                                   sigma.v0,sigma.v1,sigma.v2,sigma.v3,sigma.v4,
                                                                   rho.v01,rho.v02,rho.v03,rho.v04,
                                                                   rho.v12,rho.v13,rho.v14,
                                                                   rho.v23,rho.v24,
                                                                   rho.v34,
                                                                   alpha,
                                                                   side.test,
                                                                   Opt.Method))
  toc(log = TRUE, quiet = TRUE)
  log.txt =  tic.log(format = TRUE)
  log.lst = tic.log(format = FALSE)
  tic.clearlog()
  timings.sim.power.simulation = unlist(lapply(log.lst, function(x) x$toc - x$tic))
  
  # Get a vector with the iterations that converge
  errors = rep(0,R)
  for (r in 1:R){errors[r] = length(fit.list.sim[[r]])}
  
  R.converge = which(errors>1)
  
  # Number of replicates that converge
  n.R = length(R.converge)
  
  # Estimates the fixed effects
  beta.hat.lme.list = matrix(unlist(lapply(R.converge, function(r) fit.list.sim[[r]]$beta.hat.lme)), 
                             ncol=5, byrow=TRUE)
  colnames(beta.hat.lme.list) =  c('Intercept','GR','PE','GR*PE','AR')
  
  beta.hat.lme = colMeans(beta.hat.lme.list)
  beta.hat.lme.se = apply(beta.hat.lme.list,2,sd)/sqrt(n.R)
  
  # Power
  power.hat.lme.list = matrix(unlist(lapply(R.converge, function(r) 
    fit.list.sim[[r]]$power.hat.lme)), 
    ncol=5, byrow=TRUE)
  colnames(power.hat.lme.list) =  c('Intercept','GR','PE','GR*PE','AR')
  
  power.hat.lme = colMeans(power.hat.lme.list)
  power.hat.lme.se = sqrt(power.hat.lme*(1-power.hat.lme))/sqrt(n.R)
  
  # Standard errors
  StdError.beta.hat.lme.list = matrix(unlist(lapply(R.converge, function(r) 
    fit.list.sim[[r]]$StdError.beta.lme)), 
    ncol=5, byrow=TRUE)
  colnames(StdError.beta.hat.lme.list) = c('Intercept','GR','PE','GR*PE','AR')
  
  StdError.beta.hat.lme = colMeans(StdError.beta.hat.lme.list)
  StdError.beta.hat.lme.se = apply(StdError.beta.hat.lme.list,2,sd)/sqrt(n.R)
  
  # P-value
  p.value.beta.hat.lme.list = matrix(unlist(lapply(R.converge, function(r) 
    fit.list.sim[[r]]$p.value)), 
    ncol=5, byrow=TRUE)
  colnames(StdError.beta.hat.lme.list) =  c('Intercept','GR','PE','GR*PE','AR')
  
  return(list(beta.hat.lme.list=beta.hat.lme.list,
              beta.hat.lme=beta.hat.lme,
              beta.hat.lme.se=beta.hat.lme.se,
              power.hat.lme.list=power.hat.lme.list,
              power.hat.lme=power.hat.lme,
              power.hat.lme.se=power.hat.lme.se,
              StdError.beta.hat.lme=StdError.beta.hat.lme,
              StdError.beta.hat.lme.se=StdError.beta.hat.lme.se,
              p.value.beta.hat.lme.list=p.value.beta.hat.lme.list,
              timings.sim.power.simulation=timings.sim.power.simulation,
              n.R=n.R))}

## Power Estimation

In [160]:
set.seed(1) # Set seed or the random number generator
# Function for computing power using simulation-based approach using REML
sim.power.list = Power.Simulation.Estimates.ML.5.VAR(N, T.obs,
                                                     b00, b01, b10, b11, b20,
                                                     sigma,
                                                     sigma.v0, sigma.v1, sigma.v2, sigma.v3, sigma.v4,
                                                     rho.v01, rho.v02, rho.v03, rho.v04,
                                                     rho.v12, rho.v13, rho.v14,
                                                     rho.v23, rho.v24,
                                                     rho.v34,
                                                     alpha,
                                                     side.test,
                                                     Opt.Method="REML",
                                                     R)


# Output function
sim.power.list

# Estimated statistical power
sim.power.list$power.hat.lme


Error in solve.default(pdMatrix(a, factor = TRUE)) : 
  system is computationally singular: reciprocal condition number = 1.10929e-30
Error in solve.default(pdMatrix(a, factor = TRUE)) : 
  system is computationally singular: reciprocal condition number = 1.56806e-29
Error in solve.default(pdMatrix(a, factor = TRUE)) : 
  system is computationally singular: reciprocal condition number = 2.00429e-18
Error in solve.default(pdMatrix(a, factor = TRUE)) : 
  system is computationally singular: reciprocal condition number = 6.37469e-99
Error in solve.default(pdMatrix(a, factor = TRUE)) : 
  system is computationally singular: reciprocal condition number = 1.04748e-31
Error in solve.default(pdMatrix(a, factor = TRUE)) : 
  system is computationally singular: reciprocal condition number = 7.76435e-28
Error in solve.default(pdMatrix(a, factor = TRUE)) : 
  system is computationally singular: reciprocal condition number = 1.12613e-34


Intercept,GR,PE,GR*PE,AR
50.24350,0.094134502,0.073112401,0.1674467,-0.081349998
49.90324,0.026568512,0.031457636,0.1637441,-0.008000653
51.68086,0.039487197,0.092266834,0.1591051,-0.024002322
50.86557,-0.028024369,-0.046032390,0.1615016,0.037403982
50.42156,0.032094940,0.064358730,0.1668987,-0.014810891
50.04785,0.004945490,0.080441902,0.1649470,-0.035064959
51.52514,-0.004685288,0.001918489,0.1665235,0.005969662
52.87634,0.035523544,-0.003709060,0.1651519,-0.007662960
55.66138,-0.044165520,-0.104841175,0.1622228,0.064409653
49.84469,0.053675867,0.054061867,0.1605708,-0.044719349


Intercept         GR         PE      GR*PE         AR 
1.00000000 0.00000000 0.03030303 1.00000000 0.00000000

In [144]:
length(Y_previous)

[1] 1537

## Some Details
Dive into the data structure of one dataset

In [89]:
N = N

# Create variables days, beeps per day and Z
data.IL = expand.grid(Time=1:T.obs)

# Create variable subjno
subjno = expand.grid(1:T.obs,1:N)[,2]
data.IL = cbind(subjno,data.IL)

# Simulate error level-1 (population-level error)
# this is the line that's being used
E = rnorm(T.obs*N,0,sigma)

# Simulate error level-2
# Simulate between-subject random effect
# build the variance-covariance matrix
Sigma.v = diag(c(sigma.v0^2,sigma.v1^2,sigma.v2^2,sigma.v3^2,sigma.v4^2))
Sigma.v[lower.tri(Sigma.v)] = c(rho.v01*sigma.v0*sigma.v1,
                                rho.v02*sigma.v0*sigma.v2,
                                rho.v03*sigma.v0*sigma.v3,
                                rho.v04*sigma.v0*sigma.v4,
                                rho.v12*sigma.v1*sigma.v2,
                                rho.v13*sigma.v1*sigma.v3,
                                rho.v14*sigma.v1*sigma.v4,
                                rho.v23*sigma.v2*sigma.v3,
                                rho.v24*sigma.v2*sigma.v4,
                                rho.v34*sigma.v3*sigma.v4)
Sigma.v[upper.tri(Sigma.v)] = c(rho.v01*sigma.v0*sigma.v1,
                                rho.v02*sigma.v0*sigma.v2,
                                rho.v03*sigma.v0*sigma.v3,
                                rho.v04*sigma.v0*sigma.v4,
                                rho.v12*sigma.v1*sigma.v2,
                                rho.v13*sigma.v1*sigma.v3,
                                rho.v14*sigma.v1*sigma.v4,
                                rho.v23*sigma.v2*sigma.v3,
                                rho.v24*sigma.v2*sigma.v4,
                                rho.v34*sigma.v3*sigma.v4)

if (eigen(Sigma.v)$values[2] <= 0) {stop('The covariance matrix of the level-2 errors must be positive definite')}

# simulate the participant-wise coefficents
V.i = mvrnorm(N,rep(0,ncol(Sigma.v)),Sigma.v)

V = matrix(0,T.obs*N,ncol(V.i))

#true parameters
for (i in 1:N){
  V[which(data.IL$subjno==i),] = matrix(rep(V.i[i,], each = T.obs), ncol = T.obs)
}

# Set parameters and variables
B00 = rep(b00,nrow(data.IL))
B01 = rep(b01,nrow(data.IL))
B10 = rep(b10,nrow(data.IL))
B11 = rep(b11,nrow(data.IL))
B20 = rep(b20,nrow(data.IL))


# Simulate Dependent Variable
Y = rep(NA, length(subjno))
Y_previous = rep(NA, length(subjno))
Y_previous[sapply(1:N, function(x) which(subjno == x)[1])] = 0

for (i in 1:length(subjno)){
  Y[i] = B00[i] + B01[i]*outcome.absolute[i] + B10[i]*expectedness[i] + B11[i]*outcome.absolute[i]*expectedness[i] +
    B20[i]*Y_previous[i] + V[i,1] + V[i,2]*outcome.absolute[i] + V[i,3]*expectedness[i] + V[i,4]*outcome.absolute[i]*expectedness[i]
    + V[i,5]*Y_previous[i] + E[i]
  if (is.na(Y_previous[i+1])) {
    Y_previous[i+1] = Y[i]
  }else{}
}
                  
# Create a data frame
data = data.frame(cbind(data.IL,Y,expectedness,outcome.absolute,outcome.dummy,outcome.true,Y_previous[1:length(subjno)]))

## Fit a Hierarchical Model on this Dataset

In [95]:
fit.lme = try(lme(Y ~ outcome.absolute + expectedness + outcome.absolute*expectedness, random = ~ 1 + outcome.absolute + expectedness + outcome.absolute*expectedness|subjno,correlation = corAR1(),data=data,na.action=na.omit,method='ML',
                      control=lmeControl(opt='optim')),silent = FALSE)

In [96]:
summary(fit.lme)

Linear mixed-effects model fit by maximum likelihood
  Data: data 
       AIC      BIC    logLik
  6103.616 6189.007 -3035.808

Random effects:
 Formula: ~1 + outcome.absolute + expectedness + outcome.absolute * expectedness | subjno
 Structure: General positive-definite, Log-Cholesky parametrization
                              StdDev      Corr                
(Intercept)                    0.04695560 (Intr) otcm.b expctd
outcome.absolute               0.01742237  0.009              
expectedness                   0.20555801  0.038  0.066       
outcome.absolute:expectedness  0.12582079 -0.057  0.005 -0.012
Residual                      18.70167987                     

Correlation Structure: AR(1)
 Formula: ~1 | subjno 
 Parameter estimate(s):
      Phi 
0.9957419 
Fixed effects:  Y ~ outcome.absolute + expectedness + outcome.absolute * expectedness 
                                 Value Std.Error   DF   t-value p-value
(Intercept)                   73.83728  5.690232 1525 12.97614

In [97]:
coef(summary(fit.lme))[,"Value"]

(Intercept)              outcome.absolute 
                   73.8372815                     1.5669931 
                 expectedness outcome.absolute:expectedness 
                   -0.9594206                     0.1039099

In [98]:
coef(summary(fit.lme))[,'Std.Error']

(Intercept)              outcome.absolute 
                    5.6902317                     0.6111798 
                 expectedness outcome.absolute:expectedness 
                    0.9696793                     0.6131338

In [99]:
pt(coef(summary(fit.lme))[,4], coef(summary(fit.lme))[,3], lower = FALSE)

(Intercept)              outcome.absolute 
                 6.696187e-37                  5.222981e-03 
                 expectedness outcome.absolute:expectedness 
                 8.386928e-01                  4.327234e-01